In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [ ]:
# Column Lists

title_col = []
location_col = []
price_col = []
bedroom_col = []
bathroom_col = []
toilet_col = []
property_type_col = []
link_col = []

print("Starting scrapper...")

#RENT PROPERTIES IN AJAH - Pages 1 t 64

print("Scraping rental properties...")
for page in range(1,65):
  url = f"https://nigeriapropertycentre.com/for-rent?keywords=ajah&page={page}"

  try:
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(url, timeout=15, headers=headers)

    soup = BeautifulSoup(response.text, "lxml")
    property_listing = soup.find("div", {"itemtype": "https://schema.org/ItemList"})

    #TITLES
    titles = property_listing.find_all("h3", {"itemprop": "name"})
    for title in titles:
      title_col.append(title.text.strip())

    #LOCATIONS
    addresses = property_listing.find_all("address", class_ = "voffset-bottom-10")
    for address in addresses:
      location_col.append(address.strong.text.strip())

    #PRICES
    prices = property_listing.find_all("span", class_="price")
    for i in range(1, len(prices), 2):
      price_col.append(f"₦ {prices[i].text} per annum")

    #ROOMS (bedroom, bathroom, toilet)
    rooms_container = property_listing.find_all("ul", class_="aux-info")
    for container in rooms_container:
      room_items = container.find_all("li")

      #Bedroom
      bedroom = room_items[0].text.strip()
      bedroom_col.append(bedroom)

      #Bathroom and Toilet
      if "Bathroom" in container.text or "Bathrooms" in container.text:
        bathroom = room_items[1].text.strip()
        bathroom_col.append(bathroom)
        if "Toilet" in container.text or "Toilets" in container.text:
          toilet = room_items[2].text.strip()
          toilet_col.append(toilet)
        else:
          toilet_col.append("N/A")
      else:
        bathroom_col.append("N/A")
        if "Toilet" in container.text or "Toilets" in container.text:
          toilet = room_items[1].text
          toilet_col.append(toilet)
        else:
          toilet_col.append("N/A")

    #PROPERTY TYPE
    property_type_container = property_listing.find_all("h4", class_="content-title")
    for property_type in property_type_container:
      property_type_col.append(property_type.text.strip())

    #LINKS
    links_container = property_listing.find_all("div", class_="description")
    for links in links_container:
      link = links.a["href"]
      link_col.append(f"https://nigeriapropertycentre.com{link}")
    
    time.sleep(1)
  except requests.Timeout:
        print(f"✗ Rent page {page} timed out, skipping...")
        continue
  except requests.ConnectionError:
        print(f"✗ Rent page {page} connection error, skipping...")
        continue
  except Exception as e:
        print(f"✗ Rent page {page} error: {e}, skipping...")
        continue


# SALE PROPERTIES IN AJAH - Pages 1 to 200

for page in range(1,201):
  url = f"https://nigeriapropertycentre.com/for-sale?keywords=ajah&page={page}"

  try:
    response = requests.get(url, timeout=15, headers=headers)

    soup = BeautifulSoup(response.text, "lxml")
    
    property_listing = soup.find("div", {"itemtype": "https://schema.org/ItemList"})

    #TITLES
    titles = property_listing.find_all("h3", {"itemprop": "name"})
    for title in titles:
      title_col.append(title.text.strip())

    #LOCATIONS
    addresses = property_listing.find_all("address", class_ = "voffset-bottom-10")
    for address in addresses:
      location_col.append(address.strong.text.strip())

    #PRICES
    prices = property_listing.find_all("span", class_="price")
    for i in range(1, len(prices), 2):
      price_col.append(f"₦ {prices[i].text.strip()}")

    #ROOMS
    rooms_container = property_listing.find_all("ul", class_="aux-info")
    for container in rooms_container:
      room_items = container.find_all("li")
      bedroom = room_items[0].text.strip()
      bedroom_col.append(bedroom)

      if "Bathroom" in container.text or "Bathrooms" in container.text:
        bathroom = room_items[1].text.strip()
        bathroom_col.append(bathroom)
        if "Toilet" in container.text or "Toilets" in container.text:
          toilet = room_items[2].text.strip()
          toilet_col.append(toilet)
        else:
          toilet_col.append("N/A")
      else:
        bathroom_col.append("N/A")
        if "Toilet" in container.text or "Toilets" in container.text:
          toilet = room_items[1].text.strip()
          toilet_col.append(toilet)
        else:
          toilet_col.append("N/A")

    #PROPERTY TYPE
    property_type_container = property_listing.find_all("h4", class_="content-title")
    for property_type in property_type_container:
      property_type_col.append(property_type.text.strip())

    #LINKS
    links_container = property_listing.find_all("div", class_="description")
    for links in links_container:
      link = links.a["href"]
      link_col.append(f"https://nigeriapropertycentre.com{link}")
    time.sleep(1)
  except requests.Timeout:
      print(f"✗ Sale page {page} timed out, skipping...")
      continue
  except requests.ConnectionError:
      print(f"✗ Sale page {page} connection error, skipping...")
      continue
  except Exception as e:
      print(f"✗ Sale page {page} error: {e}, skipping...")
      continue

print(f"Scraping completed in {elapsed_time:.2f} seconds.")

#Create DataFrame
df = pd.DataFrame({
   "Title": title_col,
   "Location": location_col,
   "Price": price_col,
   "Bedrooms": bedroom_col,
   "Bathrooms": bathroom_col,
   "Toilets": toilet_col,
   "Property Type": property_type_col,
   "URL": link_col
 }, index=range(1, len(title_col)+1))

df.to_csv("ajah_properties.csv", index=True)



Starting scrapper...
Scraping rental properties...
✗ Rent page 24 connection error, skipping...
✗ Rent page 41 connection error, skipping...
✗ Rent page 48 connection error, skipping...
✗ Rent page 50 connection error, skipping...
✗ Rent page 57 connection error, skipping...
✗ Rent page 61 connection error, skipping...
✗ Sale page 7 connection error, skipping...
✗ Sale page 19 timed out, skipping...
✗ Sale page 28 connection error, skipping...
✗ Sale page 74 connection error, skipping...
✗ Sale page 86 connection error, skipping...
✗ Sale page 87 connection error, skipping...
✗ Sale page 88 connection error, skipping...
✗ Sale page 96 connection error, skipping...
✗ Sale page 110 timed out, skipping...
✗ Sale page 173 connection error, skipping...
✗ Sale page 175 connection error, skipping...
✗ Sale page 183 connection error, skipping...
✗ Sale page 189 connection error, skipping...
Scraping completed in 1598.99 seconds.


In [11]:
dataset_1 = pd.read_csv("../data/Island_Luxury_Data.csv")
dataset_2 = pd.read_csv("../data/Lekki_Central_Data.csv")
dataset_3 = pd.read_csv("../data/ajah_properties.csv")

In [12]:
dataset_1.columns = ["Title", "Location", "Price", "Bedrooms", "Bathrooms", "Toilets", "Property Type", "URL"]
dataset_2.columns = ["Title", "Location", "Price", "Bedrooms", "Bathrooms", "Toilets", "Property Type", "URL"]
dataset_3 = dataset_3.drop(columns=dataset_3.columns[-1])
dataset_3.columns = ["Title", "Location", "Price", "Bedrooms", "Bathrooms", "Toilets", "Property Type", "URL"]

merged_dataset = pd.concat([dataset_1, dataset_2, dataset_3])
merged_dataset

,Title,Location,Price,Bedrooms,Bathrooms,Toilets,Property Type,URL
0,Fully Serviced 3-bedroom Apartment With Study ...,"Old Ikoyi, Ikoyi, Lagos","₦40,000,000 per annum",3,3,4,3 bedroom flat / apartment for rent,https://nigeriapropertycentre.com/for-rent/fla...
1,Top Rated Luxury 2 Bedroom Penthouse Apartment...,"Old Ikoyi Lagos, Ikoyi, Lagos","$40,000 per annum \napprox. ₦56,842,415",2,2,3,2 bedroom flat / apartment for rent,https://nigeriapropertycentre.com/for-rent/fla...
2,Luxury 3 Bedroom Apartment,"Ikoyi, Lagos","₦35,000,000 per annum",3,4,4,3 bedroom house for rent,https://nigeriapropertycentre.com/for-rent/hou...
3,Luxury 3 Bedroom Apartment 4th Floor,"Banana Island, Ikoyi, Lagos","₦45,000,000 per annum",3,4,4,3 bedroom house for rent,https://nigeriapropertycentre.com/for-rent/hou...
4,Well Maintained Luxury 4 Bedroom Apartment,"Buena Vista, Banana Island, Ikoyi, Lagos","₦55,000,000 per annum",4,4,5,4 bedroom flat / apartment for rent,https://nigeriapropertycentre.com/for-rent/fla...
...,...,...,...,...,...,...,...,...
5073,5074,4 Bedroom Terraced Duplex With Bq,"Lekki Gardens, Ajah, Lagos","₦ 120,000,000",4 Bedrooms,4 Bathrooms,5 Toilets,4 bedroom detached duplex for sale
5074,5075,2 Plots Out Of My Land,"Abraham Adesanya, Ajah, Lagos","₦ 100,000,000",NaN,NaN,NaN,Land for sale
5075,5076,2 Plots Of Land Size Measuring 1250sqm,"By Abraham Adesanya, Ajah, Lagos","₦ 240,000,000","1,250 sqm Total Area",NaN,NaN,Land for sale
5076,5077,Plots Of Land,"Lakowe, Ajah, Ibeju Lekki, Lagos","₦ 15,000,000",500 sqm Total Area,NaN,NaN,Mixed-use land for sale


In [16]:
print(merged_dataset.duplicated().sum())

0


In [14]:
merged_dataset.to_csv("../data/Merged_dataset.csv", index=False)